Import necessary packages

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing, svm
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
scaler = StandardScaler()
from sklearn.model_selection import train_test_split, GridSearchCV

Upload cleaned data - data already cleaned and pre-processed using R files

In [7]:
X_full = pd.read_csv(r"C:\Users\wille\Documents\AppliedStats\Project\Data\X.csv")
X_full = scaler.fit_transform(X_full)
X_violent = pd.read_csv(r"C:\Users\wille\Documents\AppliedStats\Project\Data\X_refined_violent.csv", index_col=0).values
X_n_violent = pd.read_csv(r"C:\Users\wille\Documents\AppliedStats\Project\Data\X_refined_n_violent.csv", index_col=0).values
Y = pd.read_csv(r"C:\Users\wille\Documents\AppliedStats\Project\Data\Y.csv", index_col = 0)
target_violent = Y.Total_Violent.values
target_n_violent = Y["Non.Violent"].values

In [8]:
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=2101)

Create Random Forest Model

In [9]:
mse_scores = []
rf_model = RandomForestRegressor(random_state=2101)

Create grid search object for hyper-parameter tuning

In [15]:
rf_tuning_grid = {
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [None, 10, 15, 20],
    'min_samples_split': [ 2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_tuning_grid, cv=k_folds, scoring='neg_mean_squared_error')

Now, perform the grid search and fit the best model for each data split

In [ ]:
for train_index, test_index in kf.split(X_violent):
    X_train, X_test = X_violent[train_index], X_violent[test_index]
    y_train, y_test = target_violent[train_index], target_violent[test_index]

    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_rf_model = RandomForestRegressor(random_state=2101, **best_params)

    #Fit the best model
    best_rf_model.fit(X_train, y_train)


    # Make predictions on the test data
    y_pred = best_rf_model.predict(X_test)

    # Calculate accuracy and store it
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)

# Calculate the average accuracy across all folds
rf_rv = np.sqrt(np.mean(mse_scores))

In [ ]:
print("Best MSE: ", rf_rv)

Now, do the same with an XGBoost model

In [ ]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(random_state=2101)
xgb_tuning_grid = {
   'n_estimators': [100, 200, 500],
    'max_depth': [4, 6],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}
xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=xgb_tuning_grid, cv=k_folds, scoring='neg_mean_squared_error')

In [289]:
mse_scores = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_full):
    X_train, X_test = X_full[train_index], X_full[test_index]
    y_train, y_test = target_violent[train_index], target_violent[test_index]

    xgb_grid_search.fit(X_train, y_train)
    best_params = xgb_grid_search.best_params_
    best_xgb_model = XGBRegressor(random_state=2101, **best_params)

    #Fit the best model
    best_xgb_model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = best_xgb_model.predict(X_test)

    # Calculate accuracy and store it
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)

# Calculate the average accuracy across all folds
xgb_mse = np.sqrt(np.mean(mse_scores))

In [290]:
print("XGB best MSE: ", xgb_mse)

378.28901811910174

Finally, do the same with Light GBM

In [183]:
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(random_state=2101)
lgbm_tuning_grid = {
   'n_estimators': [100, 200, 500],
    'max_depth': [4, 6],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}
lgbm_grid_search = GridSearchCV(estimator=lgbm_model, param_grid=lgbm_tuning_grid, cv=k_folds, scoring='neg_mean_squared_error')

In [326]:
mse_scores = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_full):
    X_train, X_test = X_full[train_index], X_full[test_index]
    y_train, y_test = target_violent[train_index], target_violent[test_index]

    lgbm_grid_search.fit(X_train, y_train)
    best_params = lgbm_grid_search.best_params_
    best_lgbm_model = LGBMRegressor(random_state=2101, **best_params)

    #Fit the best model
    best_lgbm_model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = best_lgbm_model.predict(X_test)

    # Calculate accuracy and store it
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)

# Calculate the average accuracy across all folds
lgbm_mse = np.sqrt(np.mean(mse_scores))

In [327]:
print("LGBM best MSE: ", lgbm_mse)

370.7586193420597

Repeat for non-violent tree-based prediction .... results compiled in seperate files and presented in report